Qwen

In [1]:
from unsloth import FastModel
import torch
from torch.nn.utils import clip_grad_value_
import torch.nn as nn
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=16,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.lm_head.load_state_dict(torch.load("../../Model/qwen16.pth"))
model.model.embed_tokens.load_state_dict(torch.load("../../Model/qwen16.pth"))
model.train();
model.lm_head.weight.requires_grad_(True);
model.model.embed_tokens.weight.requires_grad_(True);

In [3]:
# resize model vocab
# model.model.embed_tokens = Gemma3TextScaledWordEmbedding(16,2560,0,50.59644256269407)
# model.lm_head = nn.Linear(2560, 16, bias=False)
# model.lm_head.weight = model.model.embed_tokens.weight

In [4]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

In [5]:
# check for causality
# inputs = next(iter(data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)))
# with torch.no_grad():
#     with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#         out = model(inputs['input_tokens'], 
#                     # attention_mask=attention_mask,
#                     )
# logits1 = out.logits
# print(inputs['input_tokens'][0][256])
# inputs['input_tokens'][0][256] += 1
# print(inputs['input_tokens'][0][256])
# with torch.no_grad():
#     with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#         out = model(inputs['input_tokens'], 
#                     # attention_mask=attention_mask,
#                     )
# logits2 = out.logits
# print(torch.where(torch.max(torch.abs(logits1 - logits2)[0],dim=1).values == 0))

#### Fine-tune embedding

In [4]:
epochs = 10
accumulation_steps = 32
lr = 3e-5
clip = 3e-3
MAX_LEN = 3600

In [5]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [6]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,inputs in enumerate(data_gen(data, IsTrain=True, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            x = inputs['input_tokens']
            y = inputs['target_tokens']
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,inputs in enumerate(data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                x = inputs['input_tokens']
                y = inputs['target_tokens']
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss: 3.4176 - Val Loss: 2.4418
Epoch 2 - Train Loss: 2.7080 - Val Loss: 2.0298
Epoch 3 - Train Loss: 2.3134 - Val Loss: 1.7909
Epoch 4 - Train Loss: 2.0285 - Val Loss: 1.6150
Epoch 5 - Train Loss: 1.7829 - Val Loss: 1.4896
Epoch 6 - Train Loss: 1.6052 - Val Loss: 1.3842
Epoch 7 - Train Loss: 1.4631 - Val Loss: 1.2873
Epoch 8 - Train Loss: 1.3674 - Val Loss: 1.2065
Epoch 9 - Train Loss: 1.2760 - Val Loss: 1.1333
Epoch 10 - Train Loss: 1.1872 - Val Loss: 1.0766
Training completed in 111.79 minutes


In [7]:
torch.save(model.lm_head.state_dict(), '../../Model/lm_heads_weights_qwen.pth')
torch.save(model.model.embed_tokens.state_dict(), '../../Model/embedding_weights_qwen.pth')

#### Fine-Tune QLORA

In [8]:
epochs = 60
accumulation_steps = 32
lr = 3e-5
clip = 3e-3
MAX_LEN = 3600

In [9]:
model.max_seq_length = MAX_LEN

In [10]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           # Larger = higher accuracy, but might overfit
    lora_alpha = 64,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [11]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [13]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,inputs in enumerate(data_gen(data, IsTrain=True, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            x = inputs['input_tokens']
            y = inputs['target_tokens']
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,inputs in enumerate(data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                x = inputs['input_tokens']
                y = inputs['target_tokens']
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

Epoch 1 - Train Loss: 0.7418 - Val Loss: 0.3818
Epoch 2 - Train Loss: 0.4534 - Val Loss: 0.3180
Epoch 3 - Train Loss: 0.3991 - Val Loss: 0.2944
Epoch 4 - Train Loss: 0.3798 - Val Loss: 0.2823
Epoch 5 - Train Loss: 0.3677 - Val Loss: 0.2746
Epoch 6 - Train Loss: 0.3580 - Val Loss: 0.2691
Epoch 7 - Train Loss: 0.3473 - Val Loss: 0.2665
Epoch 8 - Train Loss: 0.3446 - Val Loss: 0.2617
Epoch 9 - Train Loss: 0.3420 - Val Loss: 0.2589
Epoch 10 - Train Loss: 0.3368 - Val Loss: 0.2575
Epoch 11 - Train Loss: 0.3321 - Val Loss: 0.2542
Epoch 12 - Train Loss: 0.3261 - Val Loss: 0.2519
Epoch 13 - Train Loss: 0.3245 - Val Loss: 0.2499
Epoch 14 - Train Loss: 0.3224 - Val Loss: 0.2500
Epoch 15 - Train Loss: 0.3228 - Val Loss: 0.2476
Epoch 16 - Train Loss: 0.3210 - Val Loss: 0.2460
Epoch 17 - Train Loss: 0.3152 - Val Loss: 0.2455
Epoch 18 - Train Loss: 0.3164 - Val Loss: 0.2434
Epoch 19 - Train Loss: 0.3124 - Val Loss: 0.2426
Epoch 20 - Train Loss: 0.3128 - Val Loss: 0.2425
Epoch 21 - Train Loss: 0.3117

KeyboardInterrupt: 

In [19]:
for param_group in optimizer.param_groups:
    param_group['lr'] = 1.5e-5
clip = 1.5e-3

In [20]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(10):
    model.train()
    for i,inputs in enumerate(data_gen(data, IsTrain=True, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            x = inputs['input_tokens']
            y = inputs['target_tokens']
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,inputs in enumerate(data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                x = inputs['input_tokens']
                y = inputs['target_tokens']
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

Epoch 1 - Train Loss: 0.2884 - Val Loss: 0.2304
Epoch 2 - Train Loss: 0.2897 - Val Loss: 0.2302
Epoch 3 - Train Loss: 0.2880 - Val Loss: 0.2299
Epoch 4 - Train Loss: 0.2863 - Val Loss: 0.2299
Epoch 5 - Train Loss: 0.2831 - Val Loss: 0.2297


KeyboardInterrupt: 

In [4]:
epochs = 10
accumulation_steps = 32
lr = 1e-5
clip = 1e-3
MAX_LEN = 3600
model.max_seq_length = MAX_LEN

In [5]:
model = PeftModel.from_pretrained(model, "../../Model/merged_model_qwen", is_trainable=True)

In [6]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [7]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,inputs in enumerate(data_gen(data, IsTrain=True, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            x = inputs['input_tokens']
            y = inputs['target_tokens']
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,inputs in enumerate(data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                x = inputs['input_tokens']
                y = inputs['target_tokens']
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss: 0.2851 - Val Loss: 0.2289
Epoch 2 - Train Loss: 0.2846 - Val Loss: 0.2287
Epoch 3 - Train Loss: 0.2826 - Val Loss: 0.2290
Epoch 4 - Train Loss: 0.2834 - Val Loss: 0.2290
Epoch 5 - Train Loss: 0.2846 - Val Loss: 0.2287
Epoch 6 - Train Loss: 0.2846 - Val Loss: 0.2292
Epoch 7 - Train Loss: 0.2825 - Val Loss: 0.2284
Epoch 8 - Train Loss: 0.2847 - Val Loss: 0.2283
Epoch 9 - Train Loss: 0.2839 - Val Loss: 0.2288
Epoch 10 - Train Loss: 0.2812 - Val Loss: 0.2284
Training completed in 122.40 minutes


In [8]:
model.save_pretrained("../../Model/merged_model_qwen")
torch.save(model.lm_head.state_dict(), '../../Model/lm_heads_weights_qwen.pth')
torch.save(model.model.model.embed_tokens.state_dict(), '../../Model/embedding_weights_qwen.pth')

/home/zhenlan/anaconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


#### Decoding

In [ ]:
model = PeftModel.from_pretrained(model, "../../Model/merged_model_qwen", is_trainable=True)
FastLanguageModel.for_inference(model);

In [ ]:
decoder = CausalDecoder(model, IsDebug=False)

In [ ]:
best_results, any_results = [], []
for inputs_t in data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=True):
    decoder.decode(inputs_t,False);
    targets = inputs_t['target_tokens']
    a,b = check(decoder,targets)
    best_results.append(a)
    any_results.append(b)
    print(a,b)
print(f"Best results: {sum(best_results)/len(best_results)}")
print(f"Any results: {sum(any_results)/len(any_results)}")

KeyboardInterrupt: 